In [221]:
import pandas as pd
import numpy as np

In [222]:
pd.set_option('display.max_columns', None)

In [223]:
def get_data():
    cgm_data = pd.read_csv('./CGMData.csv', low_memory=False, usecols=['Date', 'Time', 'Sensor Glucose (mg/dL)'])
    insulin_data = pd.read_csv('./InsulinData.csv', low_memory=False)

    cgm_data['DateTime'] = pd.to_datetime(cgm_data['Date'] + ' ' + cgm_data['Time'])
    insulin_data['DateTime'] = pd.to_datetime(insulin_data['Date'] + ' ' + insulin_data['Time'])

    auto_mode_start = insulin_data[insulin_data['Alarm'].str.contains("AUTO MODE ACTIVE PLGM OFF", na=False)].iloc[-1]['DateTime']

    manual_data = cgm_data[cgm_data['DateTime'] < auto_mode_start]
    auto_data = cgm_data[cgm_data['DateTime'] >= auto_mode_start]

    return manual_data, auto_data

def analyze_df(data):
    data['Sensor Glucose (mg/dL)'] = pd.to_numeric(data['Sensor Glucose (mg/dL)'], errors='coerce').interpolate(limit_direction='both')
    
    data.set_index('DateTime', inplace=True)

    metrics = {
        'overnight_hyperglycemia': [],
        'overnight_hyperglycemia_critical': [],
        'overnight_in_range': [],
        'overnight_in_range_secondary': [],
        'overnight_hypoglycemia_lvl1': [],
        'overnight_hypoglycemia_lvl2': [],
        'daytime_hyperglycemia': [],
        'daytime_hyperglycemia_critical': [],
        'daytime_in_range': [],
        'daytime_in_range_secondary': [],
        'daytime_hypoglycemia_lvl1': [],
        'daytime_hypoglycemia_lvl2': [],
        'whole_day_hyperglycemia': [],
        'whole_day_hyperglycemia_critical': [],
        'whole_day_in_range': [],
        'whole_day_in_range_secondary': [],
        'whole_day_hypoglycemia_lvl1': [],
        'whole_day_hypoglycemia_lvl2': [],
    }

    daytime_start = '06:00:00'
    nighttime_start = '00:00:00'
    nighttime_end = "05:59:59"
    daytime_end = '23:59:59'
    
    for day, day_data in data.groupby(data.index.date):
        if day_data.empty:
            continue

        overnight_data = day_data.between_time(nighttime_start, nighttime_end)['Sensor Glucose (mg/dL)']
        daytime_data = day_data.between_time(daytime_start, daytime_end)['Sensor Glucose (mg/dL)']
        whole_day_data = day_data['Sensor Glucose (mg/dL)']
        
        hyperglycemia_condition = lambda x: (x > 180) & (x <= 250)
        hyperglycemia_critical_condition = lambda x: (x > 250)
        in_range_condition = lambda x: (x >= 70) & (x <= 180)
        in_range_secondary_condition = lambda x: (x >= 70) & (x <= 150)
        hypoglycemia_lvl1_condition = lambda x: (x >= 54) & (x < 70)
        hypoglycemia_lvl2_condition = lambda x: (x < 54)

        metrics['overnight_hyperglycemia'].append(hyperglycemia_condition(overnight_data).sum() / 288 * 100)
        metrics['overnight_hyperglycemia_critical'].append(hyperglycemia_critical_condition(overnight_data).sum() / 288 * 100)
        metrics['overnight_in_range'].append(in_range_condition(overnight_data).sum() / 288 * 100)
        metrics['overnight_in_range_secondary'].append(in_range_secondary_condition(overnight_data).sum() / 288 * 100)
        metrics['overnight_hypoglycemia_lvl1'].append(hypoglycemia_lvl1_condition(overnight_data).sum() / 288 * 100)
        metrics['overnight_hypoglycemia_lvl2'].append(hypoglycemia_lvl2_condition(overnight_data).sum() / 288 * 100)

        metrics['daytime_hyperglycemia'].append(hyperglycemia_condition(daytime_data).sum() / 288 * 100)
        metrics['daytime_hyperglycemia_critical'].append(hyperglycemia_critical_condition(daytime_data).sum() / 288 * 100)
        metrics['daytime_in_range'].append(in_range_condition(daytime_data).sum() / 288 * 100)
        metrics['daytime_in_range_secondary'].append(in_range_secondary_condition(daytime_data).sum() / 288 * 100)
        metrics['daytime_hypoglycemia_lvl1'].append(hypoglycemia_lvl1_condition(daytime_data).sum() / 288 * 100)
        metrics['daytime_hypoglycemia_lvl2'].append(hypoglycemia_lvl2_condition(daytime_data).sum() / 288 * 100)

        metrics['whole_day_hyperglycemia'].append(hyperglycemia_condition(whole_day_data).sum() / 288 * 100)
        metrics['whole_day_hyperglycemia_critical'].append(hyperglycemia_critical_condition(whole_day_data).sum() / 288 * 100)
        metrics['whole_day_in_range'].append(in_range_condition(whole_day_data).sum() / 288 * 100)
        metrics['whole_day_in_range_secondary'].append(in_range_secondary_condition(whole_day_data).sum() / 288 * 100)
        metrics['whole_day_hypoglycemia_lvl1'].append(hypoglycemia_lvl1_condition(whole_day_data).sum() / 288 * 100)
        metrics['whole_day_hypoglycemia_lvl2'].append(hypoglycemia_lvl2_condition(whole_day_data).sum() / 288 * 100)
    mean_metrics = {k: np.mean(v) for k, v in metrics.items()}
    return pd.Series(mean_metrics)

def output(result_df):
    result_df.columns = ['Manual Mode', 'Auto Mode']
    ordered_metrics = [
        'overnight_hyperglycemia', 'overnight_hyperglycemia_critical', 'overnight_in_range', 'overnight_in_range_secondary',
        'overnight_hypoglycemia_lvl1', 'overnight_hypoglycemia_lvl2', 'daytime_hyperglycemia', 'daytime_hyperglycemia_critical',
        'daytime_in_range', 'daytime_in_range_secondary', 'daytime_hypoglycemia_lvl1', 'daytime_hypoglycemia_lvl2',
        'whole_day_hyperglycemia', 'whole_day_hyperglycemia_critical', 'whole_day_in_range', 'whole_day_in_range_secondary',
        'whole_day_hypoglycemia_lvl1', 'whole_day_hypoglycemia_lvl2'
    ]
    result_df = result_df.loc[ordered_metrics].T
    result_df.to_csv('./Result.csv', header=False, index=False)

def main():
    manual_data, auto_data = get_data()

    manual_metrics = analyze_df(manual_data)
    auto_metrics = analyze_df(auto_data)
    result_df = pd.concat([manual_metrics, auto_metrics], axis=1)
    output(result_df)

In [224]:
main()